# Velogames solver: Tour De France 2022 (Julia Edition)
A script to calculate the optimal team that could have been chosen for a given race in [Velogames fantasy cycling](https://www.velogames.com/)

This Julia script uses the [Gumbo](https://github.com/JuliaWeb/Gumbo.jl) and [Cascadia](https://github.com/Algocircle/Cascadia.jl) libraries to scrape rider data, and the [JuMP](https://jump.dev/JuMP.jl/stable/) optimisation library / [HiGHS](https://highs.dev/) solver to construct and solve a mixed-integer program (MIP) described below

In Velogames fantasy cycling, you must select a team of 9 riders, each with a specific cost based on their expected performance, spending no more than 100 points. 

Each rider is classed as either an All-Rounder, a Climber, a Sprinter or is Unclassed. A team must contain 2 All-Rounders, 2 Climbers, 1 Sprinter and 3 Unclassed riders. The 9th selection can be from any of these categories.

At the end of the race, each rider will have accumulated a score based on their performance, and the aim is to pick a team with the highest combined score at the end of the race.

The optimisation problem can be stated as:

$maximise \sum_{j=1}^{n} x_j y_j$

$s.t.$ 

$\sum_{j=1}^{n} x_j=9$

$\sum_{j=1}^{n} x_j z_j \leq 100$

$\sum_{j=1}^{n} x_j a_j \geq 2$

$\sum_{j=1}^{n} x_j c_j \geq 2$

$\sum_{j=1}^{n} x_j s_j \geq 1$

$\sum_{j=1}^{n} x_j u_j \geq 3$

where $j=1...n$ is the set of all riders

$x_j\in[0,1]$ is a binary decision variable denoting if rider $j$ is chosen (1 for chosen, 0 for not chosen)

$z_j\in Z^+$ and $y_j\in Z^+$ are the cost and score parameters of rider $j$ respectively

$a_j\in[0,1]$, $c_j\in[0,1]$, $s_j\in[0,1]$ and $u_j\in[0,1]$ are binary parameters denoting if rider $j$ is an All-Rounder, Climber, Sprinter or Unclassed respectively, with the further parameter constraint that $a_i+c_i+s_i+u_i=1$ $\forall i=1...n$ (i.e. each rider is allocated to one and only one of the 4 categories) and by implication $\sum_{j=1}^{n} a_j+\sum_{j=1}^{n} c_j+\sum_{j=1}^{n} s_j+\sum_{j=1}^{n} u_j=n$ (i.e. the sum of the number of riders in each category is equal to the total number of riders)

In [1]:
using Gumbo
using Cascadia
using DataFrames
using JuMP
using HiGHS

In [2]:
# download rider data from webpage
url = "https://www.velogames.com/velogame/2024/riders.php"
page = parsehtml(read(download(url), String))
rider_table = eachmatch(sel"table", page.root)[1]
rider_names = String[]
rider_classes = String[]
rider_costs = Int64[]
rider_scores = Int64[]

for rider_row in eachmatch(sel"tr", rider_table)[3:end]
    rider_cells = eachmatch(sel"td", rider_row)
    push!(rider_names, text(rider_cells[2]))
    push!(rider_classes, text(rider_cells[4]))
    push!(rider_costs, parse(Int64, text(rider_cells[5])))
    push!(rider_scores, parse(Int64, text(rider_cells[7])))
end
rider_df = DataFrame(name=rider_names, class=rider_classes, cost=rider_costs, score=rider_scores)

# normalise class data
for class in unique(rider_df.class)
    rider_df[!,class] = rider_df.class .== class
end

rider_df

Row,name,class,cost,score,All Rounder,Sprinter,Climber,Unclassed
,String,String,Int64,Int64,Bool,Bool,Bool,Bool
1,Jonas Vingegaard,All Rounder,24,2703,true,false,false,false
2,Primož Roglič,All Rounder,20,609,true,false,false,false
3,Remco Evenepoel,All Rounder,18,2619,true,false,false,false
4,Jasper Philipsen,Sprinter,16,1482,false,true,false,false
5,Carlos Rodríguez,Climber,14,1101,false,false,true,false
6,Matteo Jorgenson,All Rounder,14,1365,true,false,false,false
7,Wout Van Aert,Sprinter,14,1139,false,true,false,false
8,Juan Ayuso,All Rounder,14,543,true,false,false,false
9,Adam Yates,Climber,14,1369,false,false,true,false


In [7]:
model = Model(HiGHS.Optimizer)
@variable(model, x[rider_df.name], Bin)
@objective(model, Max, rider_df.score' * x) # maximise the total score
@constraint(model, rider_df.cost' * x <= 100) # cost must be <= 100
@constraint(model, sum(x) == 9) # exactly 9 riders must be chosen
@constraint(model, rider_df[!, "All Rounder"]' * x >= 2) # at least 2 must be all rounders
@constraint(model, rider_df[!, "Sprinter"]' * x >= 1) # at least 1 must be a sprinter
@constraint(model, rider_df[!, "Climber"]' * x >= 2) # at least 2 must be climbers
@constraint(model, rider_df[!, "Unclassed"]' * x >= 3) # at least 3 must be unclassed
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
6 rows, 175 cols, 525 nonzeros
6 rows, 159 cols, 357 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   6 rows
   159 cols (145 binary, 14 integer, 0 implied int., 0 continuous)
   357 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   50050           -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   13717.333333    13146              4.35%        0      0      0         8     0.0s

42.8% inactive integer columns, restarting
Model after restart has 5 rows, 91 cols (90 bin., 1 int., 0 impl., 0 cont.), and 207 nonzeros

         0       0         0   0.00%   13717.333333    13146

In [8]:
# total score
objective_value(model)

13482.0

In [9]:
# total cost
rider_df.cost.*value.(x).data |> sum

100.0

In [10]:
# selected riders
rider_df[!,:chosen] = value.(x).data .|>  !iszero
filter(:chosen => ==(true), rider_df)

Row,name,class,cost,score,All Rounder,Sprinter,Climber,Unclassed,chosen
,String,String,Int64,Int64,Bool,Bool,Bool,Bool,Bool
1,Jonas Vingegaard,All Rounder,24,2703,true,false,false,false,true
2,Remco Evenepoel,All Rounder,18,2619,true,false,false,false,true
3,Adam Yates,Climber,14,1369,false,false,true,false,true
4,João Almeida,All Rounder,12,1788,true,false,false,false,true
5,Biniam Girmay,Sprinter,8,1741,false,true,false,false,true
6,Derek Gee,Unclassed,8,949,false,false,false,true,true
7,Mikel Landa,Climber,8,1342,false,false,true,false,true
8,Ryan Gibbons,Unclassed,4,327,false,false,false,true,true
9,Anthony Turgis,Unclassed,4,644,false,false,false,true,true
